In [149]:
import xlwings as xw
import pandas as pd
import numpy as np

### Считывание данных о застоях и формирование сгруппированных данных по месяцам

#### Данные с названиями файлов и т.д.

In [162]:
for_TD = pd.read_excel('Файл для TD exe.xlsx', 'Параметры', index_col=0, dtype = str)
path_to_KPI_file = for_TD.loc["Путь к файлу KPI", "Значение"]
path_to_plan_time_file = for_TD.loc["Путь к файлу Daily", "Значение"]
for_TD_machine_names = pd.read_excel('Файл для TD exe.xlsx', 'Названия станков', dtype = str)

#Словари для соответствия названий станков
machine = dict((str(for_TD_machine_names["Названия станков на листе downtime from CPMS"].values[i]), str(for_TD_machine_names["Названия станков в файле Daily"].values[i])) for i in range(len(for_TD_machine_names)))
machine_in_rus = dict((str(for_TD_machine_names["Названия станков в файле Daily"].values[i]), str(for_TD_machine_names["Названия станков на листе KPI"].values[i])) for i in range(len(for_TD_machine_names)))
machine_in_rus_keys = list(machine_in_rus.keys())
list(for_TD_machine_names['Названия станков в файле Daily'])

['ASAHI', '924', 'RDCGoepfert', '1232', 'BOBST', '616', 'Corrugator', 'Tanabe']

#### Чтение и группировка данных

In [163]:
# TableDowntime = pd.read_excel(path_to_KPI_file, sheet_name=for_TD.loc["Имя листа downtime from CPMS", "Значение"], usecols=['Станок', 'Код простоя', 'Месяц', 'Число', 'Начало простоя', 'Конец простоя', 'простой в мин.', 'Проблема', 'Узел'])
TableDowntime = pd.read_excel(path_to_KPI_file, sheet_name=for_TD.loc["Имя листа downtime from CPMS", "Значение"],
                    usecols=list(map(lambda x: x.strip(), for_TD.loc["Название колонок в файле KPI", "Значение"].split(','))))
TableDowntime.columns = ['Станок', 'Код простоя', 'Месяц', 'Число', 'Начало простоя', 'Конец простоя','простой в мин.', 'Проблема', 'Узел']


TableDowntime['Начало простоя'] = pd.to_datetime(TableDowntime['Начало простоя'])
TableDowntime['Конец простоя'] = pd.to_datetime(TableDowntime['Конец простоя']) #pandas понимает формат Apr, Mar и т.д.

#Разделяем простои длинной в несколько дней на несколько записей или если простой затрагивает 2 дня
#Пример 16.06 - период с 7-00 16.06.24 до 7-00 17.06.24
#Временно смещаем время на 7 часов, чтобы корректно вычислять число и месяц в формате с 7 до 7
TableDowntime['Начало простоя'] = pd.to_datetime(TableDowntime['Начало простоя'], format='%d %m %Y %H:%M:%S') - pd.Timedelta(hours=7)
TableDowntime['Конец простоя'] = pd.to_datetime(TableDowntime['Конец простоя'], format='%d %m %Y %H:%M:%S') - pd.Timedelta(hours=7)

#Делим запись с длинным простоем на записи с более короткими простоями по дням (в днях не учитываем график с 7 до 7, т.к. даты временно смещены на 7 часов)
def middle_of_day(d):
    return pd.to_datetime(f'{d.year}-{d.month}-{d.day} 12:00:00')

def end_of_day(d):
    d += pd.Timedelta(days=1)
    return pd.to_datetime(f'{d.year}-{d.month}-{d.day} 00:00:00')

def end_of_shift(d):
    return end_of_day(d) if (d >= middle_of_day(d)) else middle_of_day(d)

#Делим запись с длинным простоем на записи с более короткими простоями по сменам (в днях не учитываем график с 7 до 7, т.к. даты временно смещены на 7 часов)
for i in range(len(TableDowntime)): #цикл не пробегает по созданным в цикле записям
    cur_i = i
    # Пока затрагивается новая смена отрезаем запись и проверяем новую запись в таблице
    while (end_of_shift(TableDowntime.loc[cur_i, 'Начало простоя']) != end_of_shift(TableDowntime.loc[cur_i, 'Конец простоя'])):  
        new_end = end_of_shift(TableDowntime.loc[cur_i, 'Начало простоя'])
        new_series = TableDowntime.loc[cur_i].copy()
        new_series.loc['Начало простоя'] = new_end
        TableDowntime.loc[cur_i, 'Конец простоя'] = new_end
        cur_i = len(TableDowntime) #задаём индекс новой записи (только что созданной)
        TableDowntime.loc[len(TableDowntime)] = new_series

#Для новых записей также определяем число и месяц
TableDowntime["Число"] = TableDowntime['Начало простоя'].dt.day 
TableDowntime["Месяц"] = TableDowntime['Начало простоя'].dt.month
TableDowntime["Год"] = TableDowntime['Начало простоя'].dt.year
TableDowntime['Смена'] = 1 + (TableDowntime['Начало простоя'] >= TableDowntime['Начало простоя'].apply(middle_of_day)) #чтобы можно было сортировать по смене

#Смещаем обратно на 7 часов
TableDowntime['Начало простоя'] = pd.to_datetime(TableDowntime['Начало простоя']) + pd.Timedelta(hours=7)
TableDowntime['Конец простоя'] = pd.to_datetime(TableDowntime['Конец простоя']) + pd.Timedelta(hours=7)
TableDowntime["простой в мин."] = ((TableDowntime['Конец простоя'] - TableDowntime['Начало простоя']).dt.total_seconds()/60).round().astype(int)
TableDowntime["Станок"] = TableDowntime["Станок"].apply(lambda x : machine[x]) 
TableDowntime

,Станок,Код простоя,Месяц,Число,Начало простоя,Конец простоя,простой в мин.,Проблема,Узел,Год,Смена
0,ASAHI,CV6b1 Нет бр. >1 часа: внепл. обслуживание,4,9,2024-04-09 19:02:15,2024-04-09 23:06:57,245,NaN,Разделитель /Брейкер,2024,2
1,924,CV6b1 Нет бр. >1 часа: внепл. обслуживание,4,23,2024-04-24 01:56:05,2024-04-24 03:02:23,66,замена прижимного шланга,Печатная секция,2024,2
2,RDCGoepfert,CV6b1 Нет бр. >1 часа: внепл. обслуживание,5,20,2024-05-20 21:31:53,2024-05-20 22:50:15,78,NaN,Подача,2024,2
3,RDCGoepfert,CV6b1 Нет бр. >1 часа: внепл. обслуживание,5,20,2024-05-20 22:53:26,2024-05-21 07:00:00,487,NaN,Подача,2024,2
4,924,CV6b1 Нет бр. >1 часа: внепл. обслуживание,5,21,2024-05-21 15:03:06,2024-05-21 16:55:26,112,ремонт прижима камеры,default equipment,2024,1
...,...,...,...,...,...,...,...,...,...,...,...
1136,924,CV101 Электрический ремонт,5,4,2024-05-04 07:00:00,2024-05-04 07:02:50,3,толкатель не едет назад,Префидер,2024,1
1137,924,CV101 Электрический ремонт,4,23,2024-04-23 19:00:00,2024-04-23 19:11:10,11,отказал датчик нанесения клея отходит провод,Клеевая система,2024,2
1138,ASAHI,CV101 Электрический ремонт,3,28,2024-03-28 07:00:00,2024-03-28 07:00:19,0,NaN,default equipment,2024,1
1139,Corrugator,C101 Электрический ремонт ГА,2,14,2024-02-14 07:00:00,2024-02-14 07:39:42,40,Двигатель для воздухадувки клеевого узла,MF-2,2024,1


In [107]:
#группировка по столбцам месяц, число и станок
grouped_Downtime = TableDowntime.groupby(['Станок', 'Смена', 'Месяц', 'Число', 'Год']).agg({'простой в мин.' : 'sum'}).reset_index()
grouped_Downtime

,Станок,Смена,Месяц,Число,Год,простой в мин.
0,1232,1,1,14,2024,60
1,1232,1,1,15,2024,12
2,1232,1,1,17,2024,21
3,1232,1,2,5,2024,45
4,1232,1,2,8,2024,98
...,...,...,...,...,...,...
581,Tanabe,2,3,17,2024,84
582,Tanabe,2,3,21,2024,32
583,Tanabe,2,4,17,2024,57
584,Tanabe,2,6,4,2024,31


### Берём время работы for maintenance

In [108]:
plan_time_dict = pd.read_excel(path_to_plan_time_file, 
    skiprows=[1,2,3,4], 
    usecols = ['Date', 'Shift', 'working time-stops for maintenance'], 
    sheet_name=['Corrugator', '616', '924', '1232', 'BOBST', 'Tanabe', 'ASAHI', 'RDCGoepfert'])
for sheet_name in plan_time_dict.keys():
    plan_time_dict[sheet_name] = plan_time_dict[sheet_name].dropna()
    #Изменяем 07-19, 19-07 на 1 и 2 смену
    plan_time_dict[sheet_name]["Shift"] = plan_time_dict[sheet_name]["Shift"].apply(lambda s : 1 if (s =="07-19") else 2)

c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppDa

### Cопоставляем время работы for maintenance с записями о застоях

In [109]:
def get_plan_time(machine, shift, day, month, year):
    date = pd.to_datetime(f'{year}-{month}-{day}')
    try:
        return plan_time_dict[machine][(plan_time_dict[machine]["Date"] == date) & (plan_time_dict[machine]["Shift"] == shift)]["working time-stops for maintenance"].iloc[0]
    except IndexError:
        return np.nan
grouped_Downtime["Время работы for maintenance"] = grouped_Downtime.apply(lambda row: get_plan_time(row['Станок'], row['Смена'], row['Число'], row['Месяц'], row['Год']), axis=1)
grouped_Downtime["TD"] = grouped_Downtime["простой в мин."] / grouped_Downtime["Время работы for maintenance"] / 60 * 100 
grouped_Downtime["Дата"] = grouped_Downtime.apply(lambda row : pd.to_datetime(f'{row["Год"]}-{row["Месяц"]}-{row["Число"]}'), axis=1)
#Возвращаем смену в исходный формат
grouped_Downtime["Смена"] = grouped_Downtime["Смена"].apply(lambda n : "07-19" if (n ==1) else "19-07")
grouped_Downtime = grouped_Downtime.sort_values(by='Дата',  ascending=False)
grouped_Downtime

,Станок,Смена,Месяц,Число,Год,простой в мин.,Время работы for maintenance,TD,Дата
446,Corrugator,07-19,6,23,2024,720,NaN,NaN,2024-06-23
82,1232,19-07,6,23,2024,81,NaN,NaN,2024-06-23
522,Corrugator,19-07,6,23,2024,111,NaN,NaN,2024-06-23
563,RDCGoepfert,19-07,6,23,2024,14,NaN,NaN,2024-06-23
521,Corrugator,19-07,6,22,2024,720,0.0,inf,2024-06-22
...,...,...,...,...,...,...,...,...,...
37,1232,19-07,1,7,2024,184,NaN,NaN,2024-01-07
210,924,19-07,1,7,2024,101,NaN,NaN,2024-01-07
132,616,19-07,1,7,2024,55,NaN,NaN,2024-01-07
265,ASAHI,19-07,1,7,2024,80,NaN,NaN,2024-01-07


### Открываем файл для записи

In [110]:
KPI_book = xw.Book(path_to_KPI_file)
KPI_sheet = KPI_book.sheets[for_TD.loc["Имя листа KPI", "Значение"]]

### Раскраска по целевому KPI

In [111]:
#Считываем целевой KPI
target_KPI = pd.DataFrame(KPI_sheet.range("C10:C17").value, columns=['Целевой KPI'], index=KPI_sheet.range("B10:B17").value)

def is_in_target(row):
    return row['TD'] <= target_KPI.loc[machine_in_rus[row['Станок']]]

def colorize(df, cell_tuple):
    green = (226, 239, 218)
    red = (252, 228, 214)
    is_in_target_ser = df.apply(is_in_target, axis = 1)
    i = 0
    for c in KPI_sheet.range(f'{cell_tuple[0]+str(cell_tuple[1])}:{cell_tuple[0]+str(cell_tuple[1]+len(is_in_target_ser)-1)}'):
        if (is_in_target_ser.iloc[i]["Целевой KPI"]):
            c.color = green
        else:
            c.color = red
        i+=1

### Запись по сменам

In [112]:
#Записываем в файл группированное время
KPI_sheet.range('A132').value = grouped_Downtime[["Станок", "Смена", "Дата", "TD"]]
KPI_sheet.range('A133').expand('down').value = None #чистим индексы

### Проверки

In [113]:
#проверка
# test_book = xw.Book('Проверка.xlsx')
# test_sheet = test_book.sheets["План"]
# test_sheet.range("A1").value = plan_time_dict["Corrugator"]

In [114]:
#проверка
# get_plan_time('Corrugator', 1 ,27, 3, 2024)
# grouped_Downtime.loc[grouped_Downtime["Дата"] == pd.to_datetime('2024-01-08'), :].sort_values(by="Станок")

In [115]:
#проверка
# test_sheet.range("N1").value = grouped_Downtime[grouped_Downtime["Станок"] == "Corrugator"].loc[ : , ['Смена','Месяц','Число','Год', 'Время работы for maintenance']]

### Отчистка

In [116]:
# Отчистка
# def clear_range(ran):
#     ran.value = None
#     ran.color = None
# clear_range(KPI_sheet.range('B132').expand('table'))
# clear_range(KPI_sheet.range('H132').expand('table'))
# clear_range(KPI_sheet.range('M132').expand('table'))
# clear_range(KPI_sheet.range('D10:D17'))


In [117]:
# Отчистка
# def clear_range(ran):
#     ran.value = None
#     ran.color = None
# clear_range(KPI_sheet.range('B132').expand('table'))
# clear_range(KPI_sheet.range('H132').expand('table'))
# clear_range(KPI_sheet.range('M132').expand('table'))
# clear_range(KPI_sheet.range('D10:D17'))


### Статистика за вчера

In [118]:
#Выводим вчерашнюю статистику
today = pd.to_datetime(KPI_sheet.range('C6').value) - pd.Timedelta("1 day")
day_grouped_Downtime = grouped_Downtime.groupby(['Станок', 'Дата']).agg({"Время работы for maintenance": 'sum', "простой в мин." : "sum"}).reset_index()

day_grouped_Downtime["TD"] = day_grouped_Downtime["простой в мин."] / (day_grouped_Downtime["Время работы for maintenance"] * 60) * 100
today_TD = day_grouped_Downtime.loc[day_grouped_Downtime["Дата"] == today, ['Станок', 'TD']]
today_TD.set_index('Станок', inplace=True) #inplace - вместо возвращения нового датафрейма измениться текущий
for machine_name in (set(machine_in_rus_keys) - set(today_TD.index)): #Для машин по которым данных не было заполняем нулями
    today_TD.loc[machine_name] = 0
sorted_today_TD = np.array(today_TD.loc[['Corrugator', '616', '924', '1232', 'BOBST', 'ASAHI', 'RDCGoepfert', 'Tanabe']]["TD"])  #сортировка прихотливым индексированием

KPI_sheet.range('D10').value = sorted_today_TD.reshape(len(sorted_today_TD), 1)

### По дням

In [119]:
day_grouped_Downtime = grouped_Downtime.groupby(['Станок', 'Дата']).agg({"Время работы for maintenance": 'sum', "простой в мин." : "sum"}).reset_index()
day_grouped_Downtime["TD"] = day_grouped_Downtime["простой в мин."] / (day_grouped_Downtime["Время работы for maintenance"] * 60) * 100
day_grouped_Downtime = day_grouped_Downtime.sort_values(by='Дата',  ascending=False)
KPI_sheet.range('G132').value = day_grouped_Downtime[["Станок", "Дата", "TD"]]
KPI_sheet.range('G133').expand('down').value = None #чистим индексы

### По месяцам

In [120]:
# По Месяцам
month_grouped_Downtime = grouped_Downtime.groupby(['Станок', 'Год', 'Месяц']).agg({"Время работы for maintenance": 'sum', "простой в мин." : "sum"}).reset_index()
month_grouped_Downtime["TD"] = month_grouped_Downtime["простой в мин."] / (month_grouped_Downtime["Время работы for maintenance"] * 60) * 100

KPI_sheet.range('L132').value = month_grouped_Downtime[["Станок", "Год", "Месяц", "TD"]]
KPI_sheet.range('L133').expand('down').value = None #чистим индексы

### Раскраска

In [121]:
colorize(grouped_Downtime, ('E', 133))
colorize(day_grouped_Downtime, ('J', 133))
colorize(month_grouped_Downtime, ('P', 133))

### Проверка

In [122]:
#!Для проверки
# book = xw.Book('../../../Проверка.xlgrouped_Downtimesx')
# sheet = book.sheets["KPI"]
# sheet.range('A1').value = grouped_Downtime[["Станок", "Смена", "Дата", "Время работы for maintenance", "простой в мин.",  "TD"]].sort_values(by='Дата')

In [123]:
#!Для проверки

# def get_plan_time_string(machine, shift, day, month, year):
#     date = pd.to_datetime(f'{year}-{month}-{day}')
#     try:
#         ret = plan_time_dict[machine][(plan_time_dict[machine]["Date"] == date) & (plan_time_dict[machine]["Shift"] == shift)].iloc[0]
#         ret['Станок'] = machine
#         return ret
#     except IndexError:
        # return pd.Series({'Date': np.nan, 'Shift': np.nan, 'working time-stops for maintenance': np.nan, 'Станок': np.nan})
#         return pd.Series({'Date': date, 'Shift': np.nan, 'working time-stops for maintenance': np.nan, 'Станок': np.nan})




# plan_time_table = grouped_Downtime.apply(lambda row: get_plan_time_string(row['Станок'], row['Смена'], row['Число'], row['Месяц'], row['Год']), axis=1)

# # Возвращаем смену в исходный формат
# sheet.range('J1').value = plan_time_table.sort_values(by='Date')

# input_table = pd.read_excel(path_to_KPI_file, sheet_name='Downtime (dowload from CPMS)', usecols=['Станок', 'Код простоя', 'Месяц', 'Число', 'Начало простоя', 'Конец простоя', 'простой в мин.', 'Проблема', 'Узел'])
# input_table['Начало простоя'] = pd.to_datetime(input_table['Начало простоя'])
# sheet.range('S1').value = input_table.sort_values(by='Начало простоя')

# sheet.range('AT1').value = TableDowntime.sort_values(by='Начало простоя')

In [124]:
# 09.01.2024	19-07
# get_plan_time_string("ASAHI", 2, 9, 1, 2024)

In [125]:
#уникальные станки
# stanki = pd.read_excel(path_to_KPI_file, sheet_name='Downtime (dowload from CPMS)', usecols=["Станок"])
# print("{")
# for i in stanki["Станок"].unique():
    # print(f"\"{i}\": ,")
    # print(f"\"{i}\"")
# print("}")

In [147]:
# KPI_sheet.range("T2:T10").offset(1, 2).value = 1
# offset(dy, dx)
list(map(lambda x: x.strip(), for_TD.loc["Название колонок в файле KPI", "Значение"].split(',')))

['Станок',
 'Код    простоя',
 'Месяц',
 'Число',
 'Начало простоя',
 'Конец простоя',
 'простой в мин.',
 'Проблема',
 'Узел']

In [168]:
# KPI_book = xw.Book(path_to_KPI_file)
# try:
#     KPI_sheet = KPI_book.sheets["KPI1"]
# except:
#     raise FileNotFoundError(f"Не найден лист KPI1") 